In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv', index_col='id')
df

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0


Dataset Description
train.csv: A full training dataset with the following attributes:

id: unique id for a news article;
title: the title of a news article;
author: author of the news article;
text: the text of the article; could be incomplete;
label: a label that marks the article as potentially unreliable -- 
1: unreliable
0: reliable

In [3]:
# Import additional libraries needed to clean dataset
import numpy as np
import pandas as pd

import re #Regular expressions 
import nltk #Natural Language Toolkit
from nltk.corpus import stopwords #words that occur frequently e.g. a,an,the,in,etc. and removed for purposes of text-normalization
from nltk.stem.porter import PorterStemmer #to reduce the word to its root word e.g. actor to act
from sklearn.feature_extraction.text import TfidfVectorizer #IF-IDF Vectorizer converts textual data to numerical data
import string

### Read and Parse Text

In [4]:
# Verify if any missing values
df.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
# Remove data with missing values for a complete dataset
df.dropna(axis=0, inplace=True)

In [6]:
df.shape

(18285, 4)

By removing data with missing values, our dataset now has 18,285 rows as opposed to the initial number of rows of 20,800

In [7]:
# Add new column that will combine all columns to help determine fake and real news; this data will be used to train our model
df['combined_text'] = df['author']+' '+df['title']+' '+df['text']
print(df['combined_text'])



id
0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: combined_text, Length: 18285, dtype: object


In [8]:
df.head()

,title,author,text,label,combined_text
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [9]:
# Separate the data and label for modeling purposes
X = df.drop(columns='label', axis=1)
Y = df['label']

In [10]:
# Print the the combined text
X = df['combined_text']
print(X)

id
0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799    David Swanson What Keeps the F-35 Alive   Davi...
Name: combined_text, Length: 18285, dtype: object


In [11]:
# Print the label
print(Y) 

id
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64


### Remove Special Characters and Stopwords, and Stemming

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vanessaandjaviervillagomez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
ps = PorterStemmer()

# Create function to process the text
def stemming(text):
    #select all alaphbetical characters (lower and uppercase), and all other characters 
    #will be removed and replace numbers and punctuations with a whitespace
    stemmed_text = re.sub('[^a-zA-Z]', ' ', text)
    #convert letters to lowercase
    stemmed_text = stemmed_text.lower()
    #convert text to a splitted case or a list
    stemmed_text = stemmed_text.split()
    #apply stem to get the root words and remove stopwords 
    stemmed_text = [ps.stem(word) for word in stemmed_text if not word in stopwords.words('english')]
    #join all the words
    stemmed_text = ' '.join(stemmed_text)
    return stemmed_text


In [14]:
df['combined_text'] = df['combined_text'].apply(stemming)

In [15]:
print(df['combined_text'])

id
0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2        consortiumnew com truth might get fire truth m...
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson keep f aliv david swanson author...
Name: combined_text, Length: 18285, dtype: object


In [16]:
# Rename 'text' column
df.rename(columns={"text": "content"})

,title,author,content,label,combined_text
id,,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid even see comey letter...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire truth m...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...
...,...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,jerom hudson rapper trump poster child white s...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,benjamin hoffman n f l playoff schedul matchup...
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,michael j de la merc rachel abram maci said re...


In [17]:
 # Export the dataset into a csv
df.to_csv("clean_df.csv", index = False)

In [18]:
# Separate combined text and label values
X = df['combined_text'].values
Y = df['label'].values

In [19]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel 

In [20]:
print(Y)

[1 0 1 ... 0 1 1]


### TF-IDF
- Term Frequency, Inverse Document Frequency
- TF-IDF Vectorizer converts textual data to numerical data

In [21]:
# Create the transform
vectorizer = TfidfVectorizer()
# Tokenization
vectorizer.fit(X)
# Encode text as floating-point values for use as inputs in modeling algorithms (feature extraction)
X_vect = vectorizer.transform(X)


In [22]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet darrel lucu octob subscrib jason chaffetz stump american fork utah imag courtesi michael jolley avail creativ common licens apolog keith olbermann doubt worst person world week fbi director jame comey accord hous democrat aid look like also know second worst person well turn comey sent infam letter announc fbi look email may relat hillari clinton email server rank democrat relev committe hear comey found via tweet one republican committe chairmen know comey notifi republican chairmen democrat rank member hous intellig judiciari oversight committe agenc review email recent discov order see contain classifi inform long letter went oversight committe chairman jason chaffetz set polit world ablaz tweet fbi dir inform fbi learn exist email appear pertin investig case reopen jason chaffetz jasoninthehous octob cours know case comey actual say review email light unrel 